In [1]:
# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
import requests
from bs4 import BeautifulSoup
import csv
from datetime import datetime as dt

In [2]:
# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
place_code = {"東京":{"prec_no":"44", "block_no":"47662", "type":"s1"}, "大分":{"prec_no":"83", "block_no":"0808", "type":"a1"}, "弘前":{"prec_no":"31", "block_no":"0166", "type":"a1"}}
base_url = "http://www.data.jma.go.jp/obd/stats/etrn/view/daily_%s.php?prec_no=%s&block_no=%s&year=%s&month=%s&day=&view=p1"

# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
def str2float(str):
    try:
        return float(str)
    except:
        return 0.0

In [3]:
# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
today = dt.today()
day = int(today.day) -1 #前日の気温を取得するため
month = int(today.month)
year = int(today.year)

In [4]:
# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
l = []

In [5]:
today = dt.today()
day = int(today.day) - 1 #前日の気温を取得するため
month = int(today.month)
year = int(today.year)


#都市を網羅します
for place in place_code.keys():
    #最終的にデータを集めるリスト (下に書いてある初期値は一行目。つまり、ヘッダー。)
    # index = place_name.index(place)
    prec_no = place_code[place]["prec_no"]
    block_no = place_code[place]["block_no"]
    data_type = place_code[place]["type"]

    year = year

    r = requests.get(base_url%(data_type, prec_no, block_no, year, month))
    r.encoding = r.apparent_encoding

    # まずはサイトごとスクレイピング
    soup = BeautifulSoup(r.text)
    # findAllで条件に一致するものをすべて抜き出します。
    # 今回の条件はtrタグでclassがmtxになってるものです。
    rows = soup.findAll('tr',class_='mtx')

    # 表の最初の1~4行目はカラム情報なのでスライスする。(indexだから初めは0だよ)
    # 【追記】2020/3/11 申し訳ございません。間違えてました。
    if data_type == "a1":
        row = rows[3:][day]

        # 今度はtrのなかのtdをすべて抜き出します
        data = row.findAll('td')

        #１行の中には様々なデータがあるので全部取り出す。
        # ★ポイント
        # day = data[0].findAll('a')[0].string # 日にち
        date = str(year) + "/" + str(month) + "/" + str(day)
        precip_total  = data[1].string #降水量(mm)合計
        temp_ave = data[4].string #平均気温
        temp_high = data[5].string #最高気温
        temp_low = data[6].string #最低気温
        daylight = data[13].string #日照時間
        d = {"date":date, "precip_total":precip_total, "temp_ave":temp_ave, "temp_high":temp_high, "temp_low":temp_low, "daylight":daylight, "place":place}
        l.append(d)
    else:
        row = rows[4:][day-1]
        data = row.findAll('td')
        #１行の中には様々なデータがあるので全部取り出す。
        # ★ポイント
        # day = data[0].findAll('a')[0].string
        date = str(year) + "/" + str(month) + "/" + str(day)
        precip_total  = data[3].string #降水量(mm)合計
        temp_ave = data[6].string #平均気温
        temp_high = data[7].string #最高気温
        temp_low = data[8].string #最低気温
        daylight = data[16].string #日照時間
        d = {"date":date, "precip_total":precip_total, "temp_ave":temp_ave, "temp_high":temp_high, "temp_low":temp_low, "daylight":daylight, "place":place}
        l.append(d)

In [6]:
# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
df = pd.DataFrame(l)

In [7]:
# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
df

,date,daylight,place,precip_total,temp_ave,temp_high,temp_low
0,2021/2/23,9.7,東京,--,11.0,15.5,5.5
1,2021/2/23,9.2,大分,0.0,9.2,12.2,5.4
2,2021/2/23,1.7,弘前,7.5,-4.8,-2.4,-5.9


In [ ]:
# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
todays_weather = dataiku.Dataset("today_weather")
todays_weather.write_with_schema(df)